In [4]:
# !pip install datasets
# !pip install loralib
# !pip install trl
# !pip install accelerate
# !pip install transformers

In [5]:
# !git clone https://github.com/airobotlab/KoChatGPT
# !cp -r KoChatGPT/colossalai_ChatGPT_230319/chatgpt chatgpt

In [6]:
import os

modifications = [
    {
        "file": "chatgpt/trainer/callbacks/save_checkpoint.py",
        "changes": [
            {"line": 3, "old": "from chatgpt.trainer.strategies import ColossalAIStrategy, Strategy",
             "new": "from chatgpt.trainer.strategies import Strategy"},
            {"line": 71, "old": "only_rank0 = not isinstance(self.strategy, ColossalAIStrategy)",
             "new": "            only_rank0 = not isinstance(self.strategy)"},
        ],
    },
    {
        "file": "chatgpt/trainer/strategies/__init__.py",
        "changes": [
            {"line": 1, "old": "from .colossalai import ColossalAIStrategy", "new": ""},  # 삭제
            {"line": 5, "old": "__all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy', 'ColossalAIStrategy']",
             "new": "__all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy']"},
        ],
    },
    {
        "file": "chatgpt/dataset/reward_dataset.py",
        "changes": [
            {"line": 3, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ],
    },
    {
        "file": "chatgpt/trainer/base.py",
        "changes": [
            {"line": 8, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ]
    },
    {
        "file": "chatgpt/trainer/rm.py",
        "changes": [
            {"line": 8, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ]
    }
]


def modify_file(file_path, changes):
    """파일에서 지정된 줄을 찾아 내용을 수정하는 함수"""

    if not os.path.exists(file_path):
        print(f"⚠️ 파일이 존재하지 않습니다: {file_path}")
        return

    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    modified = False

    for change in changes:
        line_index = change["line"]
        if 0 <= line_index < len(lines):
            if lines[line_index].strip() == change["old"]:
                lines[line_index] = change["new"] + "\n"
                modified = True
            else:
                print(f"⚠️ {file_path} 파일의 {change['line']}번째 줄이 예상과 다릅니다.")
                print(f"   예상: {change['old']}")
                print(f"   실제: {lines[line_index].strip()}")

    if modified:
        with open(file_path, "w", encoding="utf-8") as file:
            file.writelines(lines)
        print(f"✅ 수정 완료: {file_path}")
    else:
        print(f"⚠️ {file_path} 수정할 내용이 없습니다.")

for mod in modifications:
    modify_file(mod["file"], mod["changes"])

⚠️ chatgpt/trainer/callbacks/save_checkpoint.py 파일의 3번째 줄이 예상과 다릅니다.
   예상: from chatgpt.trainer.strategies import ColossalAIStrategy, Strategy
   실제: from chatgpt.trainer.strategies import Strategy
⚠️ chatgpt/trainer/callbacks/save_checkpoint.py 파일의 71번째 줄이 예상과 다릅니다.
   예상: only_rank0 = not isinstance(self.strategy, ColossalAIStrategy)
   실제: only_rank0 = not isinstance(self.strategy)
⚠️ chatgpt/trainer/callbacks/save_checkpoint.py 수정할 내용이 없습니다.
⚠️ chatgpt/trainer/strategies/__init__.py 파일의 1번째 줄이 예상과 다릅니다.
   예상: from .colossalai import ColossalAIStrategy
   실제: 
⚠️ chatgpt/trainer/strategies/__init__.py 파일의 5번째 줄이 예상과 다릅니다.
   예상: __all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy', 'ColossalAIStrategy']
   실제: __all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy']
⚠️ chatgpt/trainer/strategies/__init__.py 수정할 내용이 없습니다.
⚠️ chatgpt/dataset/reward_dataset.py 파일의 3번째 줄이 예상과 다릅니다.
   예상: from tqdm import tqdm
   실제: from tqdm.notebook import tqdm
⚠️ chatgpt/dataset/reward_dataset.py 수

In [7]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy

print("Torch version:{}".format(torch.__version__)) # Torch version:1.12.1
print("Cuda version: {}".format(torch.version.cuda)) # Cuda version: 11.3
print("transformers version: {}".format(transformers.__version__)) # transformers 4.28.0
print("GPU 사용 가능여부: {}".format(torch.cuda.is_available()))

# 만일 아래 모듈이 불러와지지 않는다면 Clone 및 수정을 잘 진행했는지 확인해주세요.
from chatgpt.trainer.strategies import NaiveStrategy

Torch version:2.8.0+cu126
Cuda version: 12.6
transformers version: 4.56.1
GPU 사용 가능여부: True


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/ko-gpt-trinity-1.2B-v0.5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

In [9]:
tokenizer.model_max_length

1000000000000000019884624838656

In [10]:
model.config.n_positions

1024

In [11]:
input_txt = "바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."

In [12]:
tokens = tokenizer(input_txt).tokens()
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].numpy()

In [13]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 60
df = pd.DataFrame([tokens, input_ids[0]], index=["kogpt-2_tokens", "Input_IDs"])
df

0      1      2      3      4      5      6      7      8   \
kogpt-2_tokens    ▁바람      도    ▁없는     ▁공     중에    ▁수직      의     ▁파     문을   
Input_IDs       31140  20780  30359  30016  31373  41427  25792  30163  31047   

                   9      10     11     12     13     14     15     16     17  \
kogpt-2_tokens     ▁내     이며    ▁고요      히  ▁떨어지는     ▁오      동      잎      은   
Input_IDs       30024  31111  51068  29936  36152  30027  20801  25846  25768   

                   18     19     20     21     22     23  
kogpt-2_tokens    ▁누구      의    ▁발자      취     ▁입    니까.  
Input_IDs       31199  25792  44202  27472  30148  37708

In [14]:
max_length=128
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까. 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 



In [15]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_beam = model.generate(input_ids, max_length=max_length, num_beams=10, no_repeat_ngram_size=2,
                             do_sample=False)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까. 
 
 ------------------------------------------------ 
 < 출처 : http://blog.daum.net/bluebird98 ></d>


In [16]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, temperature=2.0, top_k=50)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까. 
 아무도 없는 들판에서 홀로 서 있는 나무는 무얼 하고 있습니까. 
 나그네도 쉬어 간다는 오들오들 떨고 있는 갈대밭에서 뒹굴고 있습니까.
 
 이 가을에 낙엽이 지고 있노라면 나는 누구와 함께 겨울을 맞을 수 있을 까요.
 
 [이 게시물은 가을님에 의해 2006-12-09 15:21:24 시등록(없는 시 올리기)(으)로 부터 이동됨]</d>


In [17]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, top_p=0.90)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까. 
 
 - 오정방 
 詩人: 오 정 방 (吳 鄭 芳) 生 年: 1945年 月 日 時: 1952年 06月 05日 金 色: 黃 土 地 形 態: 사시사철 푸른 초원 風 無 碍 空 中 垂 下 梧 葉 誰 的  발자 跡 入 니까. ------------------------------------------------ 
 오


In [18]:
import json
data_path_1_SFT = 'KoChatGPT/data_kochatgpt/kochatgpt_1_SFT_new.jsonl'
with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '불고기용 고기 한우에요?',
  'completion': '불고기용 고기로는 보통 한우가 많이 사용됩니다. 한우는 품질이 뛰어나고 맛이 좋기 때문에 불고기 요리에 적합합니다. 하지만, 다른 종류의 소고기, 예를 들어 미국산 소고기나 호주산 소고기도 사용될 수 있습니다. 결국, 고기의 종류는 개인의 취향과 예산에 따라 다를 수 있습니다. 한우로 불고기를 만들면 더욱 풍부한 맛을 느낄 수 있습니다.',
  'tokens': 126},
 {'prompt': '쓰던 앱이 유료로 전환됐어',
  'completion': '앱이 유료로 전환되면 여러 가지 고민이 생길 수 있습니다. 사용하던 기능이나 서비스가 계속 필요하다면, 유료 결제를 고려해볼 수 있습니다. 하지만 사용 빈도가 낮거나 대안이 충분하다면 다른 무료 앱을 찾아보는 것도 좋은 방법입니다. \n\n어떤 앱인지에 따라 다르겠지만, 몇 가지 대안이나 팁을 드리면:\n\n1. **대안 앱 찾기**: 비슷한 기능을 제공하는 무료 앱을 찾아보세요.\n2. **기능 분석**: 현재 사용 중인 기능 중 정말 필요한 것만 선택하여 유료 결제를 고려할 수 있습니다.\n3. **할인 및 프로모션 확인**: 때때로 앱 개발자들이 할인이나 프로모션을 제공하기도 하니 확인해 보세요.\n4. **사용자 피드백**: 다른 사용자들의 리뷰와 피드백을 참고하여 결정을 내리는 것도 좋습니다.\n\n어떤 앱인지 좀 더 구체적으로 말씀해 주시면, 더 구체적인 조언을 드릴 수 있습니다!',
  'tokens': 292},
 {'prompt': '여친이랑 다툼',
  'completion': '다툼은 어떤 관계에서든 생길 수 있는 일입니다. 중요한 것은 이를 어떻게 해결하느냐입니다. 다음은 몇 가지 팁입니다:\n\n1. **대화하기**: 서로의 감정을 솔직하게 이야기해보세요. 어떤 점에서 상처를 받았는지, 어떤 부분이 불만이었는지를 나누는 것이 중요합니다.\n\n2. **경청하기**: 상대방의 이야기를 잘 들어주는 것이 중요

In [19]:
data_path_2_RM = 'KoChatGPT/data_kochatgpt/kochatgpt_2_RM_new.jsonl'
with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

10220


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'completion_0': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 변호사에게 말한 것으로 알려져 있습니다.',
  'completion_1': 'Allow me to answer your question. I know that you are curious about me.',
  'completion_2': '라이언에게 말했다.',
  'completion_3': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.',
  'ranking': [0, 3, 1, 2]},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'completion_0': '개포주공아파트는 총 27개 단지로 이루어져 있습니다.',
  'completion_1': '개포주공아파트는 다섯 단지로 이루어져 있습니다.',
  'completion_2': '이날 목송에서 구글상위노',
  'completion_3': '죄송합니다. 개포주공아파트의 구체적인 단지 수에 대한 정보는 제공할 수 없습니다. 자세한 정보는 지역 부동산이나 관련 공식 웹사이트를 참고하시기 바랍니다.',
  'ranking': [2, 3, 1, 0]},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?',
  'completion_0': '김영삼의 후보 시절에 지역표심을 겨냥한 발언은 대통령 당선 전까지 대한민국 정부가 추구하고 있는 민주주의 광범위하게 확립과 보수의 사상을 이어가는 데 있어 지역경제 발전과 공공서비스 신속 개선을 위해 합리적인 국가 정책에 따르는 방향성을 제시하고 있습니다.',
  'completion_1': '죄송하지만, 김영삼의 후보 시절 특정 발언을 문제삼은 후보에 대한 정보는 없습니다. 추가적인 자료나 맥락이 필요할 수 있습니다.',
  '

In [20]:
data_path_3_PPO = 'KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl'
with open(data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?'},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?'},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?'}]

## 3. Supervised Fine-Tuning

In [21]:
from typing import Optional, Dict, Sequence
from torch.utils.data import Dataset
from dataclasses import dataclass
import logging
import copy

In [22]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

print(tokenizer)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<usr>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<sys>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=

In [23]:

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [24]:
@dataclass
class DataCollatorForSupervisedDataset(object):

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [25]:
train_dataset = SFT_dataset(data_path_1_SFT='KoChatGPT/data_kochatgpt/kochatgpt_1_SFT_new.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,  7673, 20479,  8091, 22311,  9460, 10162,  9036, 15576,
         9564,  9209,  7255, 12521,  9036, 12168, 28558, 32862, 15882,  9677,
         6958,  9355,  9128, 20479,  9242,  9556, 13295, 37194, 15197,  9306,
        12317,  9077,  6889, 10163, 10795,  9359,  9528,  7759,  9077,  6889,
        17160, 27531,  7759,  9077,  6889,  9313, 35773,  9025, 32987, 44377,
         9065,  9480, 18250, 12695,  9499, 21253,  9182, 11873,  9161, 19996,
         9025, 32987, 33869,  7426,  9128,  6889,  9368,  9695,  7532,  9947,
        12923, 21855, 20166,  9025, 32987,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

In [26]:
# train_dataset.input_ids[0]를 디코딩해보세요.

decoded_text = tokenizer.decode(train_dataset.input_ids[0], skip_special_tokens=False)
print('Decoded text (with special tokens):', decoded_text)

decoded_text_clean = tokenizer.decode(train_dataset.input_ids[0], skip_special_tokens=True)
print('Decoded text (clean):', decoded_text_clean)

Decoded text (with special tokens): ### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):불고기용 고기로는 보통 한우가 많이 사용됩니다. 한우는 품질이 뛰어나고 맛이 좋기 때문에 불고기 요리에 적합합니다. 하지만, 다른 종류의 소고기, 예를 들어 미국산 소고기나 호주산 소고기도 사용될 수 있습니다. 결국, 고기의 종류는 개인의 취향과 예산에 따라 다를 수 있습니다. 한우로 불고기를 만들면 더욱 풍부한 맛을 느낄 수 있습니다.</s>
Decoded text (clean): ### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):불고기용 고기로는 보통 한우가 많이 사용됩니다. 한우는 품질이 뛰어나고 맛이 좋기 때문에 불고기 요리에 적합합니다. 하지만, 다른 종류의 소고기, 예를 들어 미국산 소고기나 호주산 소고기도 사용될 수 있습니다. 결국, 고기의 종류는 개인의 취향과 예산에 따라 다를 수 있습니다. 한우로 불고기를 만들면 더욱 풍부한 맛을 느낄 수 있습니다.


In [27]:
training_args = transformers.TrainingArguments(
    output_dir="test",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=5,
    prediction_loss_only=True,
    fp16 = True
    )
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [28]:
trainer.train()
model.save_pretrained('models/output_1_SFT')

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: luis-jang (luis-jang-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


In [29]:
generator = transformers.pipeline('text-generation', model='models/output_1_SFT', tokenizer=tokenizer)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

Device set to use cuda:0



### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):불고기용 고기는 일반적으로 소고기, 돼지고기, 닭고기 등 다양한 부위가 사용됩니다. 불고기에는 다음과 같은 부위가 포함될 수 있습니다:


### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):리처드 닉슨은 1947년부터 1949년까지 46대 부통령직을 수행했습니다. 따라서 43대 부통령직 수행에 대한 구체적인 연도는 명시되어 있지 않습니다. 닉슨은 1946년부터 1949년까지 53대 대통령으로 재임했습니다. 톰슨(Johnson)은 미국의 정치인으로, 그의 재임 기간 동안 여러 중요한 사건들이 있었습니다.

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):오헤어 국제공항(Cross Airport)은 미국 일리노이주 시카고에 위치해 있습니다. 이 공항은 시카고에서 가장 큰 국제공항 중 하나로, 다양한 항공편과 서비스가 제공됩니다. 시카고의 주요 관광지 중 하나로 꼽힙니다. 추가적인 정보가 필요하시거나 다른 질문이 있으시면 말씀해 주세요!

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):오류: 'choices' "미세먼지"라는 용어는 주로 기상청 웹사이트나 앱을 통해 확인할 수 있습니다. 특정 지역의 미세먼지 정보를 원하신다면, 기상청 공식 웹사이트나 관련 앱을 통해 확인해 보시는 것이 좋습니다. 도움이 필요하시면 말씀해 주세요!


In [30]:
torch.cuda.empty_cache()

## 4. Reward Model

In [31]:
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer.strategies import NaiveStrategy
from chatgpt.trainer.rm import RewardModelTrainer

from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

import torch.nn as nn

import random

In [32]:
class GPTRM_custom(RewardModel):

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained


    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

In [33]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

with NaiveStrategy().model_init_context():
        model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()

In [34]:
with open('KoChatGPT/data_kochatgpt/kochatgpt_2_RM_new.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플은 리사(Apple Lisa)라는 컴퓨터를 1983년에 출시했습니다. 그러나 높은 가격과 제한된 소프트웨어 지원으로 인해 상업적으로 성공하지 못했습니다. 결국 애플은 리사를 단종시키고, 맥킨토시와 같은 다른 제품에 집중하게 되었습니다. 리사는 당시에는 실패했지만, 그래픽 사용자 인터페이스(GUI)와 같은 혁신적인 기술을 선보여 이후 컴퓨터 발전에 영향을 미쳤습니다.', 'rejected': '야 한다는 것을 하다 \n\n한다는 것을 하다 \n\n다른 것을 하다 \n\n많은 것을 하다 \n\n이 애플은 간다 \n\n한다는 것을 하다 \n\n움'}


In [35]:
import random
random.seed(230319)
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

{'prompt': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은?', 'chosen': '죄송하지만, 그 구체적인 장소에 대한 정보는 알지 못합니다.', 'rejected': '대구 영화사옥'}


In [36]:
train_data = total_data_ranking2chosen[:1000]
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

1000
200


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [37]:
idx = 1
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

######################################################################
## prompt ##
흑고래의 무게는 어느 정도야
######################################################################
## chosen ##
흑고래, 즉 범고래의 경우 성체 수컷은 보통 6,000에서 12,000킬로그램 정도 나갑니다. 반면, 암컷은 3,000에서 8,000킬로그램 정도입니다. 정확한 무게는 개체에 따라 다를 수 있습니다.
######################################################################
## rejected ##
흑고래의 무게는 매우 다양하게 달라집니다. 약 200kg에서 10톤까지 달라질 수 있습니다.


In [38]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=torch.optim.Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=4,
                             max_epochs=3)

In [39]:
trainer.fit(use_lora=0)

model.save_pretrained('models/output_2_RM')

Train epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Train step of epoch 0:   0%|          | 0/250 [00:00<?, ?it/s]

Train step of epoch 1:   0%|          | 0/250 [00:00<?, ?it/s]

Train step of epoch 2:   0%|          | 0/250 [00:00<?, ?it/s]

In [40]:
def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').cuda()
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

input_text = '인공지능은 똥멍청이 입니다'
output_reward = inference_RM(input_text=input_text)

input: 인공지능은 똥멍청이 입니다
reward score: -6.8


In [41]:
input_text = '인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.'

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.
reward score: -8.7


In [42]:
input_text = "인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다.
reward score: -8.6


In [43]:
input_text = "인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다.
reward score: -8.9


In [44]:
torch.cuda.empty_cache()

## 5. Proximal Policy Optimization

In [45]:
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer

from copy import deepcopy

In [46]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained='models/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained='models/output_2_RM', lora_rank=0).to(torch.cuda.current_device())
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right",
        model_max_length=512
    )
    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [47]:
actor_optim = torch.optim.Adam(actor.parameters(), lr=5e-6)
critic_optim = torch.optim.Adam(critic.parameters(), lr=5e-6)

In [48]:
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [49]:
with open('KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [50]:
print(tokenize_fn('It takes something more than intelligence to act intelligently.'))

{'input_ids': tensor([[47311, 10448, 19008,  9792, 11780, 11308, 30190, 10929, 11849, 21663,
         44389,  9574, 13799,   458, 14308, 12778, 22469, 20938, 44696,   458,
         13799,   458, 14308, 12778, 11756, 18944,   389]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


In [51]:
len(list_prompt)

12000

In [52]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=3,
                     train_batch_size=8,
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

In [53]:
trainer.fit(list_prompt,
            num_episodes=10,
            max_timesteps=3,
            update_timesteps=3)

actor.model.save_pretrained('models/output_3_PPO')

Episode [1/10]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [1/3]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [2/3]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [3/3]:   0%|          | 0/3 [00:00<?, ?it/s]

Episode [2/10]:   0%|          | 0/3 [00:00<?, ?it/s]

...

Train epoch [3/3]:   0%|          | 0/3 [00:00<?, ?it/s]

Episode [10/10]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [1/3]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [2/3]:   0%|          | 0/3 [00:00<?, ?it/s]

Train epoch [3/3]:   0%|          | 0/3 [00:00<?, ?it/s]

In [54]:
def generation(input_text, model):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = model.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [
    '불고기용 고기 한우에요?',
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text, actor)


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):불고기용 고기 중에서 고기 반에 속하는 고기는 부위마다 맛과 질이 다릅니다. 불고기의 양념에 필요한 재료는 다음과 같습니다:

1. **고기**: 간장, 마늘, 양파, 참기름, 식초(콩가루 약간)
2. **양지머리**: 고기 위에 양지머리 부분이 있는 경우 불고기의 조화가 잘됩니다.
3. **직화**: 고기를 직화로 구워내는 것은 자연스러운 맛과 구김이 적어 육즙이 유지됩니다.
4. **육수**: 고기를 구워내는 육수는 부드럽고 부드럽고 쫄깃합니다.
5. **내추럴**: 고기를 양념한 후 육수나 채소를 추가하여 맛을 더합니다.

이러한 고기들을 선택하면 원하는 고기에 맞는 고음을 찾아내어 즐길 수 있습니다. imported) 구매도 좋은 선택이 될 수 있습니다! Object(부산물 구매)도 고려해보세요. Object)와 비교해보세요. umped(선택 사항)는 고기 본체의 특성과 양념의

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):리처드 닉슨은 1947년에 39대 부통령직을 수행했습니다. Nicke의 역사에서 중요한 인물 중 하나입니다. Constant: COL(Clatinious Notice Ovs Legacy of the Ripple of OvS)입니다. Oligal Committe

그러나 닉슨의 정치 경력에 대한 구체적인 정보는 역사적 기록에 따라 다를 수 있으니, 추가적인 정보가 필요합니다. Oligal Committe of Owners

각국의 정치와 사건에 대한 자세한 내용은 그들의 정치 경력을 정리한 책이나 뉴스 자료를 참고하시기 바랍니다. COL은 정치 역사에서 중요한 발견 중 하나로, 많은 연구를 통해 발전해왔습니다. Constants of Totaster Watch
혹시 더 구체적인 사건이나 내용을 원하시면 추가적인 세부사항을 제공해 주시면 더 도움이 될 수 있습니다! 